Reverse_Image_Search

Various Imports that were required

In [ ]:
#ALL imports needed
from sklearn.ensemble import BaggingClassifier
import numpy as np
import pickle
from sklearn.neighbors import NearestNeighbors
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import pandas as pd
from matplotlib import pyplot as plt
import imghdr
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from skimage.feature import hog
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
with open("cifar-10-batches-py/data_batch_1", "rb") as file:
    data = pickle.load(file, encoding="latin1")

In [ ]:
data["data"].astype(float) 

In [ ]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
images = np.array(data['data'])
print(images.shape)
images = images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
first_image = images[0]
plt.imshow(first_image)
plt.axis('off')
plt.show()

Image proccessing/Feature extraction

In [ ]:
images = np.array(data['data']).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

In [ ]:
def preprocess(images):
    return images
#Will come beack to preproccessing images 

In [ ]:
def calculate_gradients(image0):
    image = image0.astype(float)
    gradient_magnitude = []
    gradient_orientation = []
    grad_x = []
    grad_y = []    
    for i in range(3):
        #image = np.mean(image0, axis = 2)
        # #print(image.shape, image[0].shape)
        sobel_x = np.diff(image[:,:,i], axis = 0)
        sobel_x = np.vstack([sobel_x,np.zeros((1,image.shape[0]))])
        #sobel_x = np.concatanate([sobel_x,])
        sobel_y = np.diff(image[:,:,i], axis = 1)
        sobel_y = np.hstack([sobel_y,np.zeros((image.shape[1],1))])
        print(sobel_x.shape,sobel_y.shape)
        gradient_magnitude.append(np.sqrt(sobel_x**2 + sobel_y**2))
        gradient_orientation.append(np.arctan2(sobel_y, sobel_x))
        grad_x.append(sobel_x)
        grad_y.append(sobel_y)
    gradient_magnitude = np.stack(gradient_magnitude, axis =2)
    gradient_orientation = np.stack(gradient_orientation, axis =2)
    grad_x = np.stack(grad_x, axis =2)
    grad_y = np.stack(grad_y, axis =2)
    return gradient_magnitude, gradient_orientation, grad_x.astype(float), grad_y.astype(float)

gradient_magnitude, gradient_orientation, sobel_x, sobel_y = calculate_gradients(images[50])
gradient_magnitude.shape, gradient_orientation.shape, sobel_x.shape, sobel_y.shape
gradient_magnitude.dtype, gradient_orientation.dtype, sobel_x.dtype, sobel_y.dtype
images[50].dtype
sobel_x

In [ ]:
image_index = 50
preprocessed_image = preprocess(images[image_index])
gradient_magnitude, gradient_orientation, sobel_x, sobel_y = calculate_gradients(preprocessed_image)

# Visualize the gradients and the original image
plt.figure(figsize=(12, 4))

plt.subplot(141)
plt.imshow(np.abs(sobel_x),cmap = 'jet')
plt.title("Absolute Value of X-Gradient")
plt.axis('off')

plt.subplot(142)
plt.imshow(np.abs(sobel_y),cmap = 'jet')
plt.title("Absolute Value of Y-Gradient")
plt.axis('off')

plt.subplot(143)
plt.imshow(gradient_magnitude, cmap = 'jet')
plt.title("Magnitude of Gradient")
plt.axis('off')

plt.subplot(144)
plt.imshow(preprocessed_image)
plt.title("Original Image")
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
def calculate_hog_features(gradient_magnitude, gradient_orientation):
    hog_features = hog(gradient_magnitude, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
    return hog_features

In [ ]:
hog_feature_vectors_r = []
hog_feature_vectors_g = []
hog_feature_vectors_b = []

In [ ]:
for image in images:
    print(image.shape)
    #gradient_magnitude_r, gradient_orientation_r, _,_ = calculate_gradients(image[:, :, 0])
    #gradient_magnitude_g, gradient_orientation_g,  _,_  = calculate_gradients(image[:, :, 1])
    #gradient_magnitude_b, gradient_orientation_b,  _,_  = calculate_gradients(image[:, :, 2])

    gradient_magnitude, gradient_orientation, _,_ = calculate_gradients(image)

    gradient_magnitude_r = gradient_magnitude[:, :, 0]
    gradient_orientation_r = gradient_orientation[:, :, 0]

    gradient_magnitude_g = gradient_magnitude[:, :, 1]
    gradient_orientation_g = gradient_orientation[:, :, 1]

    gradient_magnitude_b = gradient_magnitude[:, :, 2]
    gradient_orientation_b = gradient_orientation[:, :, 2]

    hog_features_r = calculate_hog_features(gradient_magnitude_r, gradient_orientation_r)
    hog_features_g = calculate_hog_features(gradient_magnitude_g, gradient_orientation_g)
    hog_features_b = calculate_hog_features(gradient_magnitude_b, gradient_orientation_b)
    scaler = StandardScaler()

    hog_features_r = scaler.fit_transform(hog_features_r.reshape(-1, 1)).reshape(-1)
    hog_features_g = scaler.fit_transform(hog_features_g.reshape(-1, 1)).reshape(-1)
    hog_features_b = scaler.fit_transform(hog_features_b.reshape(-1, 1)).reshape(-1)
    
    hog_feature_vectors_r.append(hog_features_r)
    hog_feature_vectors_g.append(hog_features_g)
    hog_feature_vectors_b.append(hog_features_b)

In [ ]:
hog_feature_vectors_r = np.array(hog_feature_vectors_r)
hog_feature_vectors_g = np.array(hog_feature_vectors_g)
hog_feature_vectors_b = np.array(hog_feature_vectors_b)

In [ ]:
average_hog_features = (np.array(hog_feature_vectors_r) + np.array(hog_feature_vectors_g) + np.array(hog_feature_vectors_b)) / 3.0
scalar_average = StandardScaler()
normalised_hog_features = scaler.fit_transform(average_hog_features)

print("Shape of the averages normalised: ", normalised_hog_features.shape)

Machine Learning

In [ ]:
labels = data['labels']
train_size = 8000
train_data = average_hog_features[:train_size:]
train_labels = labels[:train_size]
test_data = average_hog_features[train_size:, :]
test_labels = labels[train_size:]


SVM

In [ ]:
print(labels)

gamma = 'scale'
c_hyperparameter = 10
kernel_option = 'rbf'
svm_classifier = SVC(kernel= kernel_option, C=c_hyperparameter, gamma=gamma, probability=True)
pipeline = Pipeline([
    ('scaler', scaler),
    ('svm', svm_classifier)
])
pipeline.fit(train_data, train_labels)

In [ ]:
predictions = pipeline.predict(test_data)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)    



In [ ]:
# Calculate confusion matrix
confusion = confusion_matrix(test_labels, predictions)

#F1 scores 
f1_scores = f1_score(test_labels, predictions, average=None)
overall_f1 = f1_score(test_labels, predictions, average='weighted')

print("Confusion Matrix:")
print(confusion)

# print F1 scores 
print("F1 Scores for Each Class:")
for class_idx, class_name in enumerate(class_names):
    print(f"Class: {class_name}, F1 Score: {f1_scores[class_idx]:.2f}")
print("Overall F1 Score:", overall_f1)

Probability method

In [ ]:
#probabilities for each class label
probabilities = pipeline.predict_proba(test_data)

KNN

In [ ]:
# KNN pipeline
k_neighbors = 5  # Set the number of neighbors for KNN
knn_classifier = KNeighborsClassifier(n_neighbors=k_neighbors)
knn_pipeline = Pipeline([
    ('scaler', scaler),
    ('knn', knn_classifier)
])
knn_pipeline.fit(train_data, train_labels)

In [ ]:
predictions = pipeline.predict(test_data)
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

In [ ]:
# Calculate confusion matrix
confusion = confusion_matrix(test_labels, combined_predicted_classes)
#F1 scores
f1_scores = f1_score(test_labels, combined_predicted_classes, average=None)
overall_f1 = f1_score(test_labels, combined_predicted_classes, average='weighted')


print("Confusion Matrix:")
print(confusion)

# show F1 scores 
print("F1 Scores for Each Class:")
for class_index, class_name in enumerate(class_names):
    print(f"Class: {class_name}, F1 Score: {f1_scores[class_index]:.2f}")
print("Overall F1 Score:", overall_f1)

In [ ]:
# Predict probabilities for test data using the KNN pipeline
probabilities = knn_pipeline.predict_proba(test_data)

In [ ]:
# Calculate probabilities from both SVM and KNN models
svm_probabilities = pipeline.predict_proba(test_data)
knn_probabilities = knn_pipeline.predict_proba(test_data)
# defined weights as svm is a better method more weight was given
svm_weight = 0.7
knn_weight = 0.3
combined_probabilities = svm_probabilities * svm_weight + knn_probabilities * knn_weight


In [ ]:
# Get the predicted classes based on the combined probabilities
combined_predicted_classes = top_classes[:, -1]
accuracy = accuracy_score(test_labels, combined_predicted_classes)
print(f"Overall Accuracy: {accuracy:.2f}")

In [ ]:
# Calculate confusion matrix
confusion = confusion_matrix(test_labels, combined_predicted_classes)
#  F1 scores
f1_scores = f1_score(test_labels, combined_predicted_classes, average=None)
overall_f1 = f1_score(test_labels, combined_predicted_classes, average='weighted')

#Display all metrics
print("Confusion Matrix:")
print(confusion)
print("F1 Scores for Each Class:")
for class_name, f1 in zip(class_names, f1_scores):
    print(f"Class: {class_name}, F1 Score: {f1:.2f}")
print("Overall F1 Score:", overall_f1)

Display a search (query image) and output similar images

In [ ]:
#query image index from the testing set
query_index = np.random.randint(len(test_data))
query_image = np.reshape(data['data'][query_index], (3, 32, 32)).transpose(1, 2, 0)

#Prediction from the query image and what the query image 
query_prediction = combined_predicted_classes[query_index]
query_class_name = class_names[query_prediction]
plt.imshow(query_image)
plt.axis("off")
plt.show()
print(query_class_name)


In [ ]:
# Distnace between query image and other images and pulling the most simiar images
distances = pairwise_distances(query_image.flatten().reshape(1, -1), images.reshape(images.shape[0], -1))
closest_indices = np.argsort(distances.flatten())[1:6]

# Display the closest images
plt.figure(figsize=(15, 5))
for i, idx in enumerate(closest_indices):
    closest_image = images[idx]
    plt.subplot(1, 5, i + 1)
    plt.imshow(closest_image)
    plt.title(f"Closest Image {i + 1}")
    plt.axis("off")

plt.show()